In [3]:
import datetime
import re
import pandas as pd
import numpy as np

import altair as alt

In [4]:
def to_epiweek(cal_date: datetime.date):
    delta = cal_date - datetime.date(cal_date.year, 1, 1)
    days = delta.days
    epiweek = (days // 7) + 1
    epiweek = epiweek if epiweek < 53 else 52
    return (cal_date.year, epiweek)

assert to_epiweek(datetime.date(2020, 1, 1)) == (2020, 1)
assert to_epiweek(datetime.date(2020, 12, 31)) == (2020, 52)
assert to_epiweek(datetime.date(2021, 1, 1)) == (2021, 1)

In [5]:
hcw_data = pd.read_csv('data/hwc_hospitalisations.csv', delimiter='\t', dtype={'Epi Week': str, 'Hospitalisations': np.int64})
hcw_data = pd.concat([pd.DataFrame([{'Epi Week': f'2020.{i:02}', 'Hospitalisations': np.nan} for i in range(6, 22)]), 
                      hcw_data, 
                      pd.DataFrame([{'Epi Week': '2020.21', 'Hospitalisations': np.nan}])])
hcw_data.columns = ['epiweek', 'hcw_hospitalisations']
hcw_data = hcw_data.set_index('epiweek')
hcw_data

,hcw_hospitalisations
epiweek,
2020.06,NaN
2020.07,NaN
2020.08,NaN
2020.09,NaN
2020.10,NaN
...,...
2021.17,31.0
2021.18,28.0
2021.19,32.0


In [6]:
covid19_data = pd.read_csv('data/owid-covid-data.csv.gz', index_col=0)
covid19_data

,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
iso_code,,,,,,,,,,,,,,,,,,,,,
AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE,Africa,Zimbabwe,2021-05-23,38682.0,3.0,17.429,1586.0,0.0,0.571,2602.583,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
ZWE,Africa,Zimbabwe,2021-05-24,38696.0,14.0,17.714,1586.0,0.0,0.571,2603.525,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
ZWE,Africa,Zimbabwe,2021-05-25,38706.0,10.0,15.857,1587.0,1.0,0.571,2604.198,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571


In [7]:
sa_covid19_data = covid19_data.loc['ZAF']
# sa_covid19_data.insert(3, 'month_year', sa_covid19_data.date.apply(lambda d: pd.to_datetime(d).strftime('%b %Y')))
sa_covid19_data.insert(3, 'month_year', sa_covid19_data.date.apply(lambda d: '_'.join(d.split('-')[:2])))
sa_covid19_data.insert(4, 'epiweek', sa_covid19_data.date.apply(lambda d: '.'.join([f'{e:02}' for e in to_epiweek(datetime.datetime.strptime(d, '%Y-%m-%d').date())])))
sa_covid19_data

,continent,location,date,month_year,epiweek,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
iso_code,,,,,,,,,,,,,,,,,,,,,
ZAF,Africa,South Africa,2020-02-07,2020_02,2020.06,NaN,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,Africa,South Africa,2020-02-08,2020_02,2020.06,NaN,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,Africa,South Africa,2020-02-09,2020_02,2020.06,NaN,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,Africa,South Africa,2020-02-10,2020_02,2020.06,NaN,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,Africa,South Africa,2020-02-11,2020_02,2020.06,NaN,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZAF,Africa,South Africa,2021-05-23,2021_05,2021.21,1635465.0,2894.0,3105.286,55802.0,30.0,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,Africa,South Africa,2021-05-24,2021_05,2021.21,1637848.0,2383.0,3194.714,55874.0,72.0,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,Africa,South Africa,2021-05-25,2021_05,2021.21,1637848.0,0.0,2858.286,55976.0,102.0,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709


In [8]:
alt.Chart(sa_covid19_data.groupby('month_year', sort=False).mean().reset_index(), width=800).mark_line().encode(x=alt.X('month_year'), y=alt.Y('new_cases_smoothed'))

alt.Chart(...)

In [9]:
sa_covid19_data.columns

Index(['continent', 'location', 'date', 'month_year', 'epiweek', 'total_cases',
       'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccina

In [10]:
sa_covid19_data.date.apply(lambda d: pd.to_datetime(d).strftime('%b %Y'))

iso_code
ZAF    Feb 2020
ZAF    Feb 2020
ZAF    Feb 2020
ZAF    Feb 2020
ZAF    Feb 2020
         ...   
ZAF    May 2021
ZAF    May 2021
ZAF    May 2021
ZAF    May 2021
ZAF    May 2021
Name: date, Length: 476, dtype: object

In [11]:
sa_covid19_data.groupby('month_year', sort=False).mean().reset_index()

,month_year,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,2020_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
1,2020_03,3.767778e+02,50.111111,51.577955,2.400000,1.000000,0.077955,6.352889,0.844852,0.869682,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
2,2020_04,2.934800e+03,143.133333,123.623833,45.433333,3.266667,2.995300,49.483533,2.413367,2.084400,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
3,2020_05,1.604432e+04,872.129032,738.230387,315.451613,18.709677,15.806484,270.522290,14.704871,12.447258,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
4,2020_06,8.071570e+04,3950.866667,3457.423867,1605.933333,65.800000,61.052400,1360.942167,66.615267,58.295400,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
5,2020_07,3.244489e+05,11031.419355,10683.087484,4942.516129,172.516129,157.465355,5470.512548,186.000032,180.126871,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
6,2020_08,5.791157e+05,4318.000000,5084.728161,11605.451613,198.193548,208.649839,9764.433258,72.805484,85.733258,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
7,2020_09,6.529344e+05,1576.600000,1679.228567,15612.500000,86.166667,90.557167,11009.085333,26.582867,28.313433,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
8,2020_10,7.003533e+05,1648.806452,1597.576032,18175.516129,82.000000,85.041516,11808.611129,27.800387,26.936677,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
9,2020_11,7.542226e+05,2151.733333,2047.690467,20411.900000,75.300000,74.247633,12716.899200,36.280200,34.526000,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709


In [12]:
sa_covid19_data[sa_covid19_data.epiweek > '2020.09'].groupby(['epiweek']).mean().join(hcw_data).reset_index()

,epiweek,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,hcw_hospitalisations
0,2020.10,2.666667e+00,1.166667,1.000000,NaN,NaN,0.000000,0.045167,0.019667,0.017000,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,NaN
1,2020.11,3.814286e+01,7.857143,5.122571,NaN,NaN,0.000000,0.643143,0.132286,0.086286,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,NaN
2,2020.12,2.768571e+02,70.285714,34.102000,NaN,NaN,0.000000,4.668143,1.185000,0.575000,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,NaN
3,2020.13,1.136000e+03,114.142857,122.734714,2.400000,1.000000,0.245000,19.154000,1.924571,2.069571,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,NaN
4,2020.14,1.574571e+03,56.571429,62.653000,9.142857,1.142857,1.061286,26.549000,0.953857,1.056429,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021.17,1.576333e+06,1169.714286,1231.959286,54196.857143,48.000000,54.877571,26578.451714,19.722429,20.772000,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,31.0
61,2021.18,1.585402e+06,1547.714286,1295.591857,54473.285714,41.285714,39.489857,26731.365714,26.095857,21.845000,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,28.0
62,2021.19,1.598018e+06,2126.000000,1802.244857,54830.000000,56.000000,50.959286,26944.078286,35.846286,30.387429,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,32.0
63,2021.20,1.616136e+06,2821.571429,2588.326429,55313.142857,79.428571,69.020571,27249.569714,47.574571,43.641571,...,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709,29.0


In [13]:
data = sa_covid19_data.groupby(['epiweek']).mean().join(hcw_data).reset_index()
alt.Chart(data).mark_bar().encode(x=alt.X('epiweek:O'), y=alt.Y('new_cases_smoothed'), 
                                  color=alt.Color('positive_rate', scale=alt.Scale(range=['blue', 'green', 'yellow', 'red']), title='Case Positivity'))

alt.Chart(...)

In [14]:
data = sa_covid19_data[sa_covid19_data.epiweek > '2020.09'].groupby(['epiweek']).mean().join(hcw_data).reset_index()
base = alt.Chart(data, width=1200).encode(alt.X('epiweek:O', title='Epidemiological Week'))

cases = base.mark_bar().encode(y=alt.Y('new_cases_smoothed', title='New Cases per Day'), 
                               color=alt.Color('positive_rate', scale=alt.Scale(range=['blue', 'green', 'yellow', 'red']), title='Case Positivity'))

hcw_hospitalisations = base.mark_line().encode(y=alt.Y('hcw_hospitalisations', title='Healthcare Worker Hospitalisations per Week'))
alt.layer(cases, hcw_hospitalisations).resolve_scale(y='independent')

alt.LayerChart(...)

In [15]:
base = alt.Chart(sa_covid19_data, width=1000).encode(
    alt.X('yearmonthdate(date):T', title='epiweek:O', axis=alt.Axis(tickCount=14, labelAngle=90))
)
    
cases = base.mark_bar().encode(y=alt.Y('new_cases_smoothed', title='Daily Cases'),
                                                        color=alt.Color('positive_rate', 
                                                                        scale=alt.Scale(range=['blue', 'green', 'yellow', 'red']), title='Case Positivity'))
deaths = base.mark_line(color='red').encode(
                                                        y=alt.Y('new_deaths_smoothed', title='Daily Deaths'))
alt.layer(cases, deaths).resolve_scale(y='independent')

alt.LayerChart(...)

In [16]:
alt.Chart(sa_covid19_data, width=1000).mark_line(color='red').encode(x=alt.X('yearmonthdate(date)', title='Date', axis=alt.Axis(tickCount=14, labelAngle=90)),
                                                        y=alt.Y('new_deaths_smoothed', title='Daily Cases'))

alt.Chart(...)

## Some metrics

1. uptick: trend upwards for the past X days
2. case threshold: cases per 100,000 population
3. percentage changed compared to 7 days ago
4. test positivity 
5. tests per 100K population
6. % change in number of tests compared to 7 days ago

1, 2 and 3 derived from: https://sacmcepidemicexplorer.co.za/

Martin Prince, Kings College: https://twitter.com/MartinKGHI

In [17]:
# compute when a range is monotonically increasing
def is_increasing(arr):
    for i in range(1, len(arr)):
        if arr[i-1] >= arr[i]:
            return False
    else:
        return True

In [18]:
# test the is_increasing function
test_df = pd.DataFrame({'cases': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})
test_df = pd.DataFrame({'cases': list(range(10, 1, -1))})
test_df = pd.DataFrame({'cases': [1, 2, 3, 3, 3, 3, 7, 8, 9, 10]})
test_df.cases.rolling(5).apply(is_increasing, raw=True)


0    NaN
1    NaN
2    NaN
3    NaN
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    1.0
Name: cases, dtype: float64

In [19]:
sa_covid19_data.new_cases_smoothed.rolling(3).apply(is_increasing)

iso_code
ZAF    NaN
ZAF    NaN
ZAF    NaN
ZAF    NaN
ZAF    NaN
      ... 
ZAF    1.0
ZAF    1.0
ZAF    0.0
ZAF    0.0
ZAF    0.0
Name: new_cases_smoothed, Length: 476, dtype: float64

In [20]:
data = sa_covid19_data.copy()
data.insert(0, 'increasing', sa_covid19_data.new_cases_smoothed.rolling(3).apply(is_increasing))
data

,increasing,continent,location,date,month_year,epiweek,total_cases,new_cases,new_cases_smoothed,total_deaths,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
iso_code,,,,,,,,,,,,,,,,,,,,,
ZAF,NaN,Africa,South Africa,2020-02-07,2020_02,2020.06,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,NaN,Africa,South Africa,2020-02-08,2020_02,2020.06,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,NaN,Africa,South Africa,2020-02-09,2020_02,2020.06,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,NaN,Africa,South Africa,2020-02-10,2020_02,2020.06,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,NaN,Africa,South Africa,2020-02-11,2020_02,2020.06,NaN,NaN,NaN,NaN,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZAF,1.0,Africa,South Africa,2021-05-23,2021_05,2021.21,1635465.0,2894.0,3105.286,55802.0,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,1.0,Africa,South Africa,2021-05-24,2021_05,2021.21,1637848.0,2383.0,3194.714,55874.0,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709
ZAF,0.0,Africa,South Africa,2021-05-25,2021_05,2021.21,1637848.0,0.0,2858.286,55976.0,...,12294.876,18.9,200.38,5.52,8.1,33.2,43.993,2.32,64.13,0.709


In [36]:
alt.Chart(data, width=800).mark_bar().encode(x=alt.X('yearmonthdate(date):T', title='epiweek:O', axis=alt.Axis(tickCount=14, labelAngle=90)),
                                  y=alt.Y('new_cases_smoothed', title='Daily Cases'),
                                  color=alt.Color('increasing', title='5 Day Increase', scale=alt.Scale(range=['blue', 'red'])))
 

alt.Chart(...)